### Data Scraping
Scrape data for each season for boxscores and DraftKings fantasy salary information from Basketball-Reference and RotoGuru. Check existing files and create directories automatically.

In [1]:
import os
import re
import time
import glob

import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

from constants import DATA_DIR, SEASON_DATES, SECONDS_SLEEP, URL_BBREF, BOXSCORE_FORMAT, COLUMN_HEADERS, URL_ROTO_FORMAT

In [22]:
# Scraping Historical Game Data from Basketball-Reference.com
def get_boxscores(season, date_list):
    print("Scraping boxscores from the {} regular season".format(season))

    for date in tqdm(date_list):
        # BeautifulSoup object for a list of boxscores on a given day
        url_summaries = URL_BBREF + BOXSCORE_FORMAT.format(
            month=date[4:6], day=date[6:8], year=date[0:4]
        )
        soup_summaries = BeautifulSoup(urlopen(url_summaries), "lxml")
        games = soup_summaries.find_all("div", class_="game_summary expanded nohover")

        for game in games:
            summary = {}

            host = game.find_all("table")[1].find_all("a")[1]["href"][7:10]

            winner = game.find("tr", class_="winner").find_all("td")
            loser = game.find("tr", class_="loser").find_all("td")

            summary["winner"] = [
                winner[0].find("a")["href"][7:10],
                int(winner[1].get_text()),
            ]
            summary["loser"] = [
                loser[0].find("a")["href"][7:10],
                int(loser[1].get_text()),
            ]

            url_game = URL_BBREF + game.find("a", string="Box Score")["href"]
            soup_game = BeautifulSoup(urlopen(url_game), "lxml")

            teams = ["winner", "loser"]
            basic_stat_template = "box-{team}-game-basic"
            advanced_stat_template = "box-{team}-game-advanced"

            for team in teams:
                if summary[team][0] == host:
                    home = 1
                else:
                    home = 0

                basic_stat = basic_stat_template.format(team=summary[team][0].upper())
                advanced_stat = advanced_stat_template.format(
                    team=summary[team][0].upper()
                )

                game_data = [
                    date,
                    summary[team][0],
                    home,
                    summary["winner"][0],
                    summary["winner"][1],
                    summary["loser"][0],
                    summary["loser"][1],
                ]

                data_basic = (
                    soup_game.find("table", id=basic_stat)
                    .find("tbody")
                    .find_all("tr", class_=None)
                )
                data_advanced = (
                    soup_game.find("table", id=advanced_stat)
                    .find("tbody")
                    .find_all("tr", class_=None)
                )

                n = len(data_basic)

                player_names = [data_basic[i].find("a").get_text() for i in range(n)]

                player_data = []
                injury_keywords = ["Did Not Play", "Not With Team"]

                for i in range(n):
                    if data_basic[i].find("td").get_text() not in injury_keywords:
                        data = (
                            [player_names[i]]
                            + game_data
                            + [td.get_text() for td in data_basic[i].find_all("td")]
                            + [
                                td.get_text()
                                for td in data_advanced[i].find_all("td")[1:]
                            ]
                        )

                        player_data.append(data)

                df = pd.DataFrame(player_data, columns=COLUMN_HEADERS)
                df = df.fillna(0)
                
                df["MP"] = [
                    0.00
                    if ":" not in t
                    else round(int(t.split(":")[0]) + int(t.split(":")[1]) / 60, 2)
                    for t in df["MP"]
                ]
                df.to_csv(
                    os.path.join(
                        *[
                            DATA_DIR,
                            "Boxscores",
                            season,
                            date + "-" + summary[team][0] + ".csv",
                        ]
                    ),
                    index=False,
                )

            time.sleep(SECONDS_SLEEP)
    return None


# Scraping DraftKings salary data from RotoGuru.com
def get_fantasy_salary(season, date_list):
    print("Scraping salary information from the {} regular season".format(season))

    for date in tqdm(date_list):
        print(date)
        teams, positions, players, starters, salaries = [], [], [], [], []

        url_date = URL_ROTO_FORMAT.format(month=date[4:6], day=date[6:8], year=date[0:4])
        soup = BeautifulSoup(urlopen(url_date), "lxml")

        # Check if there were any games on a given date
        soup_table = soup.find("body").find("table", border="0", cellspacing="5")

        soup_rows = soup_table.find_all("tr")

        for row in soup_rows:
            if row.find("td").has_attr("colspan") == False:
                if row.find("a").get_text() != "":
                    position = row.find_all("td")[0].get_text()

                    player_tmp = row.find("a").get_text().split(", ")
                    player = player_tmp[1] + " " + player_tmp[0]

                    starter_tmp = row.find_all("td")[1].get_text()

                    if "^" in starter_tmp:
                        starter = 1
                    else:
                        starter = 0

                    salary_tmp = row.find_all("td")[3].get_text()
                    salary = re.sub("[$,]", "", salary_tmp)

                    team = row.find_all("td")[4].get_text()

                    positions.append(position)
                    players.append(player)
                    starters.append(starter)
                    salaries.append(salary)
                    teams.append(team)

        df = pd.DataFrame(
            {
                "Date": [date for i in range(len(players))],
                "Team": [team.upper() for team in teams],
                "Starter": starters,
                "Pos": positions,
                "Name": players,
                "Salary": salaries,
            }
        )

        df = df.loc[:, ["Date", "Team", "Pos", "Name", "Starter", "Salary"]]
        df.to_csv(
            os.path.join(DATA_DIR, "DKSalary", season, "salary_" + date + ".csv"),
            index=False,
        )

        time.sleep(SECONDS_SLEEP)
    return None

In [23]:
# Comment out season dates in SEASON_DATES in constants.py to extract data for specific seasons
for data_type in ["Boxscores", "DKSalary"]:
    for season in SEASON_DATES.keys():
        if not os.path.exists(os.path.join(DATA_DIR, data_type, season)):
            # Create a new directory and scrape the entire season
            os.mkdir(os.path.join(DATA_DIR, data_type, season))
            start_date = SEASON_DATES[season][0]
            end_date = SEASON_DATES[season][1]
            date_list = [
                d.strftime("%Y%m%d") for d in pd.date_range(start_date, end_date)
            ]

            if data_type == "Boxscores":
                get_boxscores(season, date_list)
            else:
                get_fantasy_salary(season, date_list)

        elif os.path.exists(os.path.join(DATA_DIR, data_type, season)):
            # Iterate over the existing files by name and scrape missing dates
            start_date = SEASON_DATES[season][0]
            end_date = SEASON_DATES[season][1]
            # Dates to scrape box scores from
            date_list = [
                d.strftime("%Y%m%d") for d in pd.date_range(start_date, end_date)
            ]

            if data_type == "Boxscores":
                for date in date_list:
                    # Check if csv files of the form {date}-{hometeam}.csv (i.e. 20131029-CHI.csv) exists
                    if (
                        len(
                            glob.glob(
                                os.path.join(
                                    DATA_DIR, data_type, season, str(date) + "*.csv"
                                )
                            )
                        )
                        > 0
                    ):
                        # Set back the start day by
                        date_list = date_list[date_list.index(date) :]

                get_boxscores(season, date_list)

            else:
                for date in date_list:
                    # Check if csv files of the form salary_{date}.csv (i.e. salary_20131029.csv) exists
                    if os.path.exists(
                        os.path.join(
                            DATA_DIR, data_type, season, "salary_{}.csv".format(date)
                        )
                    ):
                        date_list = date_list[date_list.index(date) :]

                get_fantasy_salary(season, date_list)

Scraping boxscores from the 2014-15 regular season


  0%|          | 0/169 [00:00<?, ?it/s]

KeyboardInterrupt: 